# Set Up Environment

In [ ]:
# Unzip image files

!!unzip "/content/gdrive/My Drive/Datasets/shared/celeba/CelebA-HQ-img.zip" -d "/content/gdrive/My Drive/Datasets/shared/celeba/CelebA-HQ-img/"

Archive:  /content/gdrive/My Drive/Datasets/shared/celeba/CelebA-HQ-img.zip


In [3]:
# set up environment with requirements.txt 

!pip install -r /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/requirements.txt

     |████████████████████████████████| 128 kB 4.1 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 61 kB 8.8 MB/s 
     |████████████████████████████████| 1.1 MB 48.3 MB/s 
     |████████████████████████████████| 11.5 MB 28.0 MB/s 
     |████████████████████████████████| 108 kB 85.5 MB/s 
     |████████████████████████████████| 14.8 MB 63.5 MB/s 
     |████████████████████████████████| 9.5 MB 14.4 MB/s 
     |████████████████████████████████| 3.1 MB 61.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 25.9 MB 5.8 kB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x4106000 @  0x7fddb3a09615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5

# Set up working directory

In [1]:
# set up working directory and mount google drive 
use_colab = True

if use_colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    #!ls "/content/gdrive/My Drive/Datasets/shared/HW4_shared_files"

    ## Update the experiments directory
    EXPERIMENTS_DIRECTORY = '/content/gdrive/My Drive/Datasets/shared/experiments/'
    DATA_DIRECTORY = '/content/gdrive/My Drive/Datasets/shared/celeba/'


Mounted at /content/gdrive


In [2]:
# change working directory

%cd /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/restyle_encoder/

/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/restyle_encoder


# Attributes 

In [ ]:
# read in attribute annotation files

attributes_file = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/CelebAMask-HQ-attribute-anno.txt'

from pandas import DataFrame, read_csv
import pandas as pd
df = pd.read_csv(attributes_file, sep='\t')

In [ ]:
print(df.columns)

Index(['filename', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')


In [ ]:
# filter all images with attribute (Not Young)
newdf = df[df['Young'] != 1]
newdf = newdf.reset_index()
print(newdf['filename'])

print(newdf['filename'][0:20])


not_young_files = []
for fn in range(len(newdf)):
  not_young_files.append(newdf['filename'][fn])

In [ ]:
# copy images with attribute to new folder

scr_folder = '/content/gdrive/MyDrive/Datasets/shared/celeba/CelebA-HQ-img/CelebA-HQ-img/'
cp_folder = '/content/gdrive/MyDrive/Datasets/shared/celeba/not_young/'
import os
import shutil
os.makedirs(cp_folder, exist_ok=True)


for file_name in not_young_files:
    # construct full file path
    source = scr_folder + file_name
    destination = cp_folder + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied', file_name)

In [ ]:
# filter all images with attribute (Heavy Makeup)
newdf = df[df['Heavy_Makeup'] == 1]

newdf = newdf.reset_index()
print(newdf['filename'])

print(newdf['filename'][0:20])

# copy images with attribute to new folder
not_young_files = []
for fn in range(len(newdf)):
  not_young_files.append(newdf['filename'][fn])

scr_folder = '/content/gdrive/MyDrive/Datasets/shared/celeba/CelebA-HQ-img/CelebA-HQ-img/'
cp_folder = '/content/gdrive/MyDrive/Datasets/shared/celeba/Heavy_Makeup/'

os.makedirs(cp_folder, exist_ok=True)
for file_name in not_young_files:
    # construct full file path
    source = scr_folder + file_name
    destination = cp_folder + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied', file_name)

# psp stylegan2 attention 

In [3]:
import os
import numpy as np
import PIL.Image
from tqdm import trange 
import argparse

from argparse import Namespace
import torch

from models.psp import pSp

In [4]:
device = torch.device("cuda")
model = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_stylegan2_attention_ffhq_500k.pt'
#mod = torch.load(model,map_location='cpu')
mod = torch.load(model,map_location='cuda:0')
mod['opts']['use_attention'] = True
print(mod['opts'])

{'exp_dir': 'experiment/psp_stylegan_attention_ffhq_encode7', 'dataset_type': 'ffhq_encode', 'encoder_type': 'GradualStyleEncoder', 'use_attention': True, 'use_gansformer': False, 'use_stylegan': False, 'input_nc': 3, 'output_size': 256, 'device': 'cuda:0', 'batch_size': 8, 'test_batch_size': 2, 'workers': 40, 'test_workers': 8, 'learning_rate': 9e-05, 'optim_name': 'ranger', 'train_decoder': False, 'start_from_latent_avg': True, 'lpips_lambda': 0.8, 'id_lambda': 0.1, 'l2_lambda': 1.0, 'w_norm_lambda': 0.0, 'lpips_lambda_crop': 0, 'l2_lambda_crop': 0, 'moco_lambda': 0, 'stylegan_weights': 'pretrained_models/stylegan2-ffhq-config-f.pt', 'checkpoint_path': 'experiment/psp_stylegan_attention_ffhq_encode6/checkpoints/best_model.pt', 'max_steps': 500000, 'image_interval': 100, 'board_interval': 50, 'val_interval': 5000, 'save_interval': 10000, 'n_iters_per_batch': 5, 'use_amp': False}


In [ ]:
for key in mod['opts'].keys():
  print(key + ": " + str(mod['opts'][key]))
  #print(mod['opts'][key])

In [6]:
#test_opts = testoptions().parse()
opts = mod['opts']
opts['latent_avg'] = mod['latent_avg']
opts['dataset_type']='ffhq_encode'
opts['resize_outputs'] = True
#opts.update(vars(testoptions().parse()))
opts = Namespace(**opts)
opts.checkpoint_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_stylegan2_attention_ffhq_500k.pt'
net = pSp(opts).to(device)
#print(net)
#net = pSps(opts,mod)

loading encoder
GradualStyleEncoder
Loading ReStyle pSp from checkpoint: /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_stylegan2_attention_ffhq_500k.pt
cuda:0


In [7]:
from tqdm import tqdm
import time
from torch.utils.data import DataLoader


from configs import data_configs
from datasets.inference_dataset import InferenceDataset

from models.psp import pSp

from utils.common import tensor2im
from utils.inference_utils import run_on_batch, get_average_image

In [8]:
data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'

#data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/CelebA-HQ-img/CelebA-HQ-img/'
exp_dir = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/experiment/famous/'
exp_path_results = os.path.join(exp_dir, 'psp_stylegan2_attention')
os.makedirs(exp_path_results, exist_ok=True)

out_path_results = os.path.join('/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/', 'famous/psp_stylegan2_attention')
os.makedirs(out_path_results, exist_ok=True)

dataset_args = data_configs.DATASETS[opts.dataset_type]
#print(dataset_args)
transforms_dict = dataset_args['transforms'](opts).get_transforms()

#dpath = os.path.join('/content/gdrive/MyDrive/Datasets/shared/celeba/CelebA-HQ-img/','CelebA-HQ-img/')
#print(transforms_dict)
#print(os.path.isdir(data_path))
dataset = InferenceDataset(root=data_path,
                           transform=transforms_dict['transform_inference'],
                           opts=opts)

##print(dataset)
dataloader = DataLoader(dataset, 
                        batch_size=opts.batch_size,
                        shuffle=False,
                        num_workers=int(4),
                        drop_last=False)
n_images = len(dataset)
print(dataloader)
avg_image = get_average_image(net, opts)
print(avg_image.size())
global_i = 0
global_time = []
all_latents = {}
#print(opts.test_workers)
resize_amount = (256, 256)
for input_batch in tqdm(dataloader):
  #print(global_i)
  if global_i >= n_images:break
  with torch.no_grad():
    input_cuda = input_batch.cuda().float()
    tic = time.time()
    result_batch, result_latents = run_on_batch(input_cuda, net, opts, avg_image)
    toc = time.time()
    global_time.append(toc - tic)
  for i in range(input_batch.shape[0]):
    results = [tensor2im(result_batch[i][iter_idx]) for iter_idx in range(opts.n_iters_per_batch)]
    im_path = dataset.paths[global_i]

    # save step-by-step results side-by-side
    
    for idx, result in enumerate(results):
      save_dir = os.path.join(out_path_results, str(idx))
      os.makedirs(save_dir, exist_ok=True)
      result.resize(resize_amount).save(os.path.join(save_dir, os.path.basename(im_path)))

    # store all latents with dict pairs (image_name, latents)
    all_latents[os.path.basename(im_path)] = result_latents[i]

    global_i += 1

stats_path = os.path.join(exp_path_results, 'stats.txt')
result_str = 'Runtime {:.4f}+-{:.4f}'.format(np.mean(global_time), np.std(global_time))
print(result_str)

with open(stats_path, 'w') as f:
    f.write(result_str)

# save all latents as npy file
np.save(os.path.join(exp_path_results, 'latents.npy'), all_latents)



torch.Size([3, 256, 256])


  0%|          | 0/19 [00:00<?, ?it/s]

  5%|▌         | 1/19 [00:04<01:12,  4.03s/it]

 11%|█         | 2/19 [00:07<01:04,  3.80s/it]

 16%|█▌        | 3/19 [00:11<00:59,  3.73s/it]

...

 84%|████████▍ | 16/19 [00:59<00:11,  3.77s/it]

 89%|████████▉ | 17/19 [01:03<00:07,  3.78s/it]

 95%|█████████▍| 18/19 [01:07<00:03,  3.79s/it]

100%|██████████| 19/19 [01:08<00:00,  3.61s/it]


Runtime 3.1954+-0.5344


# psp gansformer attention 

In [9]:

import os
from argparse import Namespace
from tqdm import tqdm
import time
import numpy as np
import torch
from torch.utils.data import DataLoader
import sys

import math
import torch
from torch import nn

from models.psp_gansformer import pSpGansformer

In [10]:
device = torch.device("cuda")
model = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_gansformer_attention_ffhq_500k.pt'
mod = torch.load(model,map_location='cuda:0')
#mod['opts']['use_attention'] = True
print(mod['opts'])

{'exp_dir': 'experiment/psp_attention_ffhq_encode8', 'dataset_type': 'ffhq_encode', 'encoder_type': 'GradualStyleEncoder', 'use_attention': False, 'use_gansformer': True, 'use_stylegan': False, 'input_nc': 3, 'output_size': 256, 'device': 'cuda:1', 'batch_size': 8, 'test_batch_size': 2, 'workers': 40, 'test_workers': 8, 'learning_rate': 9e-05, 'optim_name': 'ranger', 'train_decoder': False, 'start_from_latent_avg': True, 'lpips_lambda': 0.8, 'id_lambda': 0.1, 'l2_lambda': 1.0, 'w_norm_lambda': 0.0, 'lpips_lambda_crop': 0, 'l2_lambda_crop': 0, 'moco_lambda': 0, 'stylegan_weights': 'pretrained_models/ffhq-gansformer-256.pkl', 'checkpoint_path': 'experiment/psp_attention_ffhq_encode7/checkpoints/best_model.pt', 'max_steps': 500000, 'image_interval': 100, 'board_interval': 50, 'val_interval': 5000, 'save_interval': 10000, 'n_iters_per_batch': 5, 'use_amp': True}


In [11]:
opts = mod['opts']
opts['latent_avg'] = mod['latent_avg']
opts['dataset_type']='ffhq_encode'
opts['resize_outputs'] = True
opts['stylegan_weights']='https://drive.google.com/uc?id=1tgs-hHaziWrh0piuX3sEd8PwE9gFwlNh'
opts['device']='cuda:0'
opts = Namespace(**opts)
opts.checkpoint_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_gansformer_attention_ffhq_500k.pt'
net = pSpGansformer(opts).to(device)


Loading ReStyle pSp from checkpoint: /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_gansformer_attention_ffhq_500k.pt
https://drive.google.com/uc?id=1tgs-hHaziWrh0piuX3sEd8PwE9gFwlNh


Downloading...
From: https://drive.google.com/uc?id=1tgs-hHaziWrh0piuX3sEd8PwE9gFwlNh
To: /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/restyle_encoder/ffhq-snapshot.pkl
100%|██████████| 367M/367M [00:03<00:00, 122MB/s]


<_io.BufferedReader name='ffhq-snapshot.pkl'>
dnnlib.tflib.network
Network
numpy.core.multiarray
_reconstruct
numpy
ndarray
numpy
dtype
numpy.core.multiarray
scalar


In [12]:
from configs import data_configs
from datasets.inference_dataset import InferenceDataset

from utils.common import tensor2im
from utils.inference_utils import run_on_batch, get_average_image

In [13]:

data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'
exp_dir = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/experiment/famous/'
exp_path_results = os.path.join(exp_dir, 'psp_gansformer_attention')
os.makedirs(exp_path_results, exist_ok=True)

out_path_results = os.path.join('/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/', 'famous/psp_gansformer_attention')
os.makedirs(out_path_results, exist_ok=True)

dataset_args = data_configs.DATASETS[opts.dataset_type]
#print(dataset_args)
transforms_dict = dataset_args['transforms'](opts).get_transforms()

#dpath = os.path.join('/content/gdrive/MyDrive/Datasets/shared/celeba/CelebA-HQ-img/','CelebA-HQ-img/')
#print(transforms_dict)
#print(os.path.isdir(data_path))
dataset = InferenceDataset(root=data_path,
                           transform=transforms_dict['transform_inference'],
                           opts=opts)

##print(dataset)
dataloader = DataLoader(dataset, 
                        batch_size=opts.batch_size,
                        shuffle=False,
                        num_workers=int(4),
                        drop_last=False)
n_images = len(dataset)
print(dataloader)
avg_image = get_average_image(net, opts)
print(avg_image.size())
global_i = 0
global_time = []
all_latents = {}
#print(opts.test_workers)
resize_amount = (256, 256)
for input_batch in tqdm(dataloader):
  #print(global_i)
  if global_i >= n_images:break
  with torch.no_grad():
    input_cuda = input_batch.cuda().float()
    tic = time.time()
    result_batch, result_latents = run_on_batch(input_cuda, net, opts, avg_image)
    toc = time.time()
    global_time.append(toc - tic)
  for i in range(input_batch.shape[0]):
    results = [tensor2im(result_batch[i][iter_idx]) for iter_idx in range(opts.n_iters_per_batch)]
    im_path = dataset.paths[global_i]

    # save step-by-step results side-by-side
    
    for idx, result in enumerate(results):
      save_dir = os.path.join(out_path_results, str(idx))
      os.makedirs(save_dir, exist_ok=True)
      result.resize(resize_amount).save(os.path.join(save_dir, os.path.basename(im_path)))

    # store all latents with dict pairs (image_name, latents)
    all_latents[os.path.basename(im_path)] = result_latents[i]

    global_i += 1

stats_path = os.path.join(exp_path_results, 'stats.txt')
result_str = 'Runtime {:.4f}+-{:.4f}'.format(np.mean(global_time), np.std(global_time))
print(result_str)

with open(stats_path, 'w') as f:
    f.write(result_str)

# save all latents as npy file
np.save(os.path.join(exp_path_results, 'latents.npy'), all_latents)



Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
torch.Size([3, 256, 256])


  0%|          | 0/19 [00:00<?, ?it/s]

  5%|▌         | 1/19 [00:04<01:24,  4.68s/it]

 11%|█         | 2/19 [00:08<01:14,  4.39s/it]

 16%|█▌        | 3/19 [00:13<01:08,  4.29s/it]

...

 84%|████████▍ | 16/19 [01:08<00:13,  4.34s/it]

 89%|████████▉ | 17/19 [01:13<00:08,  4.34s/it]

 95%|█████████▍| 18/19 [01:17<00:04,  4.35s/it]

100%|██████████| 19/19 [01:19<00:00,  4.16s/it]


Runtime 3.7426+-0.6167


# PSP FFHQ encode

In [14]:
import os
import numpy as np
import PIL.Image
from tqdm import trange 
import argparse


from models.psp import pSp

In [15]:
device = torch.device("cuda")
model = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_ffhq_encode.pt'
#mod = torch.load(model,map_location='cpu')
mod = torch.load(model,map_location='cuda:0')
#mod['opts']['use_attention'] = False
print(mod['opts'])

{'exp_dir': '', 'dataset_type': 'ffhq_encode', 'encoder_type': 'GradualStyleEncoder', 'input_nc': 3, 'label_nc': 0, 'batch_size': 8, 'test_batch_size': 8, 'workers': 8, 'test_workers': 2, 'learning_rate': 0.0001, 'optim_name': 'ranger', 'train_decoder': False, 'start_from_latent_avg': True, 'learn_in_w': False, 'lpips_lambda': 0.8, 'id_lambda': 0.1, 'l2_lambda': 1.0, 'w_norm_lambda': 0, 'lpips_lambda_crop': 0, 'l2_lambda_crop': 0, 'stylegan_weights': '', 'checkpoint_path': None, 'max_steps': 300000, 'image_interval': 100, 'board_interval': 50, 'val_interval': 2500, 'save_interval': 1000, 'resize_factors': None, 'device': 'cuda:0'}


In [16]:
opts = mod['opts']
opts['use_attention'] = False
opts['latent_avg'] = mod['latent_avg']
opts['dataset_type']='ffhq_encode'
opts['resize_outputs'] = True
opts['use_stylegan'] =False
opts['output_size'] =1024
opts['n_iters_per_batch']=5
opts = Namespace(**opts)
opts.checkpoint_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_ffhq_encode.pt'
net = pSp(opts).to(device)


loading encoder
GradualStyleEncoder
Loading ReStyle pSp from checkpoint: /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/psp_ffhq_encode.pt
cuda:0


In [17]:
from argparse import Namespace
from tqdm import tqdm
import time
import torch
from torch.utils.data import DataLoader


from configs import data_configs
from datasets.inference_dataset import InferenceDataset

from utils.common import tensor2im
from utils.inference_utils import run_on_batch, get_average_image

In [18]:

data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'
exp_dir = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/experiment/famous/'
exp_path_results = os.path.join(exp_dir, 'psp_ffhq_encode')
os.makedirs(exp_path_results, exist_ok=True)


out_path_results = os.path.join('/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/', 'famous/psp_ffhq_encode/')
os.makedirs(out_path_results, exist_ok=True)

dataset_args = data_configs.DATASETS[opts.dataset_type]
#print(dataset_args)
transforms_dict = dataset_args['transforms'](opts).get_transforms()

dataset = InferenceDataset(root=data_path,
                           transform=transforms_dict['transform_inference'],
                           opts=opts)

##print(dataset)
dataloader = DataLoader(dataset, 
                        batch_size=opts.batch_size,
                        shuffle=False,
                        num_workers=int(4),
                        drop_last=False)
n_images = len(dataset)
print(dataloader)
avg_image = get_average_image(net, opts)
print(avg_image.size())
global_i = 0
global_time = []
all_latents = {}
#print(opts.test_workers)
resize_amount = (256, 256)
for input_batch in tqdm(dataloader):
  #print(global_i)
  if global_i >= n_images:break
  with torch.no_grad():
    input_cuda = input_batch.cuda().float()
    tic = time.time()
    result_batch, result_latents = run_on_batch(input_cuda, net, opts, avg_image)
    toc = time.time()
    global_time.append(toc - tic)
  for i in range(input_batch.shape[0]):
    results = [tensor2im(result_batch[i][iter_idx]) for iter_idx in range(opts.n_iters_per_batch)]
    im_path = dataset.paths[global_i]

    # save step-by-step results side-by-side
    
    for idx, result in enumerate(results):
      save_dir = os.path.join(out_path_results, str(idx))
      os.makedirs(save_dir, exist_ok=True)
      result.resize(resize_amount).save(os.path.join(save_dir, os.path.basename(im_path)))

    # store all latents with dict pairs (image_name, latents)
    all_latents[os.path.basename(im_path)] = result_latents[i]

    global_i += 1

stats_path = os.path.join(opts.exp_dir, 'stats.txt')
result_str = 'Runtime {:.4f}+-{:.4f}'.format(np.mean(global_time), np.std(global_time))
print(result_str)

with open(stats_path, 'w') as f:
    f.write(result_str)

# save all latents as npy file
np.save(os.path.join(exp_path_results, 'latents.npy'), all_latents)



torch.Size([3, 256, 256])


  0%|          | 0/19 [00:00<?, ?it/s]

  5%|▌         | 1/19 [00:04<01:27,  4.85s/it]

 11%|█         | 2/19 [00:09<01:17,  4.58s/it]

 16%|█▌        | 3/19 [00:13<01:11,  4.49s/it]

...

 84%|████████▍ | 16/19 [01:11<00:13,  4.43s/it]

 89%|████████▉ | 17/19 [01:15<00:08,  4.42s/it]

 95%|█████████▍| 18/19 [01:20<00:04,  4.43s/it]

100%|██████████| 19/19 [01:21<00:00,  4.29s/it]


Runtime 3.8628+-0.6499


# LOSS

In [29]:
from argparse import ArgumentParser
import os
import json
import sys
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd


from criteria.lpips.lpips import LPIPS
from datasets.gt_res_dataset import GTResDataset

In [26]:
def run_on_step_output(step, output_path, data_path,batch_size,workers,mode):

	transform = transforms.Compose([transforms.Resize((256, 256)),transforms.ToTensor(),transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

	step_outputs_path = os.path.join(output_path, step)

	print('Loading dataset')
	dataset = GTResDataset(root_path=step_outputs_path,
						   gt_dir=data_path,
						   transform=transform)
	#print(dataset)
	print('dataloader')
	dataloader = DataLoader(dataset,
							batch_size=batch_size,
							shuffle=False,
							num_workers=int(workers),
							drop_last=True)
	#print(dataloader)
	if mode == 'lpips':
		loss_func = LPIPS(net_type='alex')
	elif mode == 'l2':
		loss_func = torch.nn.MSELoss()
	else:
		raise Exception('Not a valid mode!')
	loss_func.cuda()

	global_i = 0
	scores_dict = {}
	all_scores = []
	losses = []
	print('loss')
	for result_batch, gt_batch in tqdm(dataloader):
		#print(result_batch)
		for i in range(batch_size):
			loss = float(loss_func(result_batch[i:i+1].cuda(), gt_batch[i:i+1].cuda()))
			all_scores.append(loss)
			im_path = dataset.pairs[global_i][0]
			scores_dict[os.path.basename(im_path)] = loss
			global_i += 1
			#print(loss)

	all_scores = list(scores_dict.values())
	#print(all_scores)
	#print(scores_dict)
	#print(scores_dict.values())
	mean = np.mean(all_scores)
	std = np.std(all_scores)
	result_str = 'Average loss is {:.4f}+-{:.4f}'.format(mean, std)
	print('Finished with ', step_outputs_path)
	print(result_str)

	out_path = os.path.join(os.path.dirname(output_path), 'inference_metrics')
	if not os.path.exists(out_path):
		os.makedirs(out_path)

	with open(os.path.join(out_path, f'stat_{mode}_step_{step}.txt'), 'w') as f:
		f.write(result_str)
	with open(os.path.join(out_path, f'scores_{mode}_step_{step}.json'), 'w') as f:
		json.dump(scores_dict, f)
	return scores_dict

In [37]:
output_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_ffhq_encode/'
data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'

batch_size = 1
workers = 4
mode = 'l2'

step='4'
step_outputs_path = os.path.join(output_path, step)
if os.path.isdir(step_outputs_path):
  print('#' * 80)
  print(f'Running on step: {step}')
  print('#' * 80)
  all_scores= run_on_step_output(step=step, output_path= output_path, data_path=data_path,batch_size=batch_size,
                      workers=workers,mode=mode)
  print(all_scores)
  print(len(all_scores))
  scores = {'img':all_scores.keys() , 
            'l2 psp base': all_scores.values()}
  df = pd.DataFrame(scores,index = list(range(len(all_scores))))
  print(df)


################################################################################
Running on step: 4
################################################################################
Loading dataset
dataloader
loss


100%|██████████| 146/146 [00:01<00:00, 103.69it/s]

Finished with  /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_ffhq_encode/4
Average loss is 0.0359+-0.0122
{'1048.jpg': 0.055495671927928925, '1053.jpg': 0.030436605215072632, '1056.jpg': 0.02354595623910427, '1123.jpg': 0.02006179466843605, '1125.jpg': 0.017132798209786415, '1140.jpg': 0.02270878106355667, '1145.jpg': 0.021233048290014267, '1146.jpg': 0.020868664607405663, '116.jpg': 0.025159034878015518, '1211.jpg': 0.0610496960580349, '1220.jpg': 0.019951729103922844, '1222.jpg': 0.02361392043530941, '1255.jpg': 0.01849798485636711, '1259.jpg': 0.054645419120788574, '1262.jpg': 0.02319708839058876, '1278.jpg': 0.023503735661506653, '1288.jpg': 0.04194425791501999, '1396.jpg': 0.029527535662055016, '1454.jpg': 0.028481177985668182, '1475.jpg': 0.05798745155334473, '1492.jpg': 0.043055277317762375, '1495.jpg': 0.03255049139261246, '1500.jpg': 0.03302421793341637, '1501.jpg': 0.044437043368816376, '1516.jpg': 0.03398198261857033, '152.jpg': 0.04840312898159027, '1558.jpg'

In [38]:
output_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_stylegan2_attention/'
data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'
batch_size = 1
workers = 4
mode = 'l2'

step='4'
step_outputs_path = os.path.join(output_path, step)
if os.path.isdir(step_outputs_path):
  print('#' * 80)
  print(f'Running on step: {step}')
  print('#' * 80)
  all_scores= run_on_step_output(step=step, output_path= output_path, data_path=data_path,batch_size=batch_size,
                      workers=workers,mode=mode)
  df['l2 psp stylegan2'] = all_scores.values()
  print(df)

################################################################################
Running on step: 4
################################################################################
Loading dataset
dataloader
loss


100%|██████████| 146/146 [00:01<00:00, 101.27it/s]

Finished with  /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_stylegan2_attention/4
Average loss is 0.0352+-0.0121
          img  l2 psp base  l2 psp stylegan2
0    1048.jpg     0.055496          0.070964
1    1053.jpg     0.030437          0.029259
2    1056.jpg     0.023546          0.022495
3    1123.jpg     0.020062          0.020812
4    1125.jpg     0.017133          0.017382
..        ...          ...               ...
141   968.jpg     0.034556          0.030363
142   981.jpg     0.066034          0.072204
143   989.jpg     0.039325          0.039463
144   990.jpg     0.031376          0.028654
145   991.jpg     0.041199          0.039777

[146 rows x 3 columns]


In [39]:
output_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_gansformer_attention/'
data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'
batch_size = 1
workers = 4
mode = 'l2'

step='4'
step_outputs_path = os.path.join(output_path, step)
if os.path.isdir(step_outputs_path):
  print('#' * 80)
  print(f'Running on step: {step}')
  print('#' * 80)
  all_scores= run_on_step_output(step=step, output_path= output_path, data_path=data_path,batch_size=batch_size,
                      workers=workers,mode=mode)
  df['l2 psp gansformer'] = all_scores.values()
  print(df)

################################################################################
Running on step: 4
################################################################################
Loading dataset
dataloader
loss


100%|██████████| 146/146 [00:01<00:00, 132.73it/s]

Finished with  /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_gansformer_attention/4
Average loss is 0.0522+-0.0174
          img  l2 psp base  l2 psp stylegan2  l2 psp gansformer
0    1048.jpg     0.055496          0.070964           0.085880
1    1053.jpg     0.030437          0.029259           0.038544
2    1056.jpg     0.023546          0.022495           0.037449
3    1123.jpg     0.020062          0.020812           0.028149
4    1125.jpg     0.017133          0.017382           0.023909
..        ...          ...               ...                ...
141   968.jpg     0.034556          0.030363           0.052896
142   981.jpg     0.066034          0.072204           0.110232
143   989.jpg     0.039325          0.039463           0.046755
144   990.jpg     0.031376          0.028654           0.033184
145   991.jpg     0.041199          0.039777           0.046131

[146 rows x 4 columns]


In [40]:
output_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_ffhq_encode/'
data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'
batch_size = 1
workers = 4
mode = 'lpips'

step='4'
step_outputs_path = os.path.join(output_path, step)
if os.path.isdir(step_outputs_path):
  print('#' * 80)
  print(f'Running on step: {step}')
  print('#' * 80)
  all_scores= run_on_step_output(step=step, output_path= output_path, data_path=data_path,batch_size=batch_size,
                      workers=workers,mode=mode)
  df['lpips psp base'] = all_scores.values()
  print(df)


################################################################################
Running on step: 4
################################################################################
Loading dataset
dataloader


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Downloading: "https://raw.githubusercontent.com/richzhang/PerceptualSimilarity/master/lpips/weights/v0.1/alex.pth" to /root/.cache/torch/hub/checkpoints/alex.pth


  0%|          | 0.00/5.87k [00:00<?, ?B/s]

loss


100%|██████████| 146/146 [00:01<00:00, 83.56it/s] 


Finished with  /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_ffhq_encode/4
Average loss is 0.1568+-0.0310
          img  l2 psp base  l2 psp stylegan2  l2 psp gansformer  \
0    1048.jpg     0.055496          0.070964           0.085880   
1    1053.jpg     0.030437          0.029259           0.038544   
2    1056.jpg     0.023546          0.022495           0.037449   
3    1123.jpg     0.020062          0.020812           0.028149   
4    1125.jpg     0.017133          0.017382           0.023909   
..        ...          ...               ...                ...   
141   968.jpg     0.034556          0.030363           0.052896   
142   981.jpg     0.066034          0.072204           0.110232   
143   989.jpg     0.039325          0.039463           0.046755   
144   990.jpg     0.031376          0.028654           0.033184   
145   991.jpg     0.041199          0.039777           0.046131   

     lpips psp base  
0          0.268026  
1          0.149615  
2       

In [41]:
output_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_stylegan2_attention/'
data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'
batch_size = 1
workers = 4
mode = 'lpips'

step='4'
step_outputs_path = os.path.join(output_path, step)
if os.path.isdir(step_outputs_path):
  print('#' * 80)
  print(f'Running on step: {step}')
  print('#' * 80)
  all_scores= run_on_step_output(step=step, output_path= output_path, data_path=data_path,batch_size=batch_size,
                      workers=workers,mode=mode)
  df['lpips psp stylegan2'] = all_scores.values()
  print(df)

################################################################################
Running on step: 4
################################################################################
Loading dataset
dataloader
loss


100%|██████████| 146/146 [00:01<00:00, 90.05it/s] 


Finished with  /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_stylegan2_attention/4
Average loss is 0.1555+-0.0309
          img  l2 psp base  l2 psp stylegan2  l2 psp gansformer  \
0    1048.jpg     0.055496          0.070964           0.085880   
1    1053.jpg     0.030437          0.029259           0.038544   
2    1056.jpg     0.023546          0.022495           0.037449   
3    1123.jpg     0.020062          0.020812           0.028149   
4    1125.jpg     0.017133          0.017382           0.023909   
..        ...          ...               ...                ...   
141   968.jpg     0.034556          0.030363           0.052896   
142   981.jpg     0.066034          0.072204           0.110232   
143   989.jpg     0.039325          0.039463           0.046755   
144   990.jpg     0.031376          0.028654           0.033184   
145   991.jpg     0.041199          0.039777           0.046131   

     lpips psp base  lpips psp stylegan2  
0          0.268026    

In [42]:
output_path = '/content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_gansformer_attention/'
data_path = '/content/gdrive/MyDrive/Datasets/shared/celeba/famous/'
batch_size = 1
workers = 4
mode = 'lpips'

step='4'
step_outputs_path = os.path.join(output_path, step)
if os.path.isdir(step_outputs_path):
  print('#' * 80)
  print(f'Running on step: {step}')
  print('#' * 80)
  all_scores= run_on_step_output(step=step, output_path= output_path, data_path=data_path,batch_size=batch_size,
                      workers=workers,mode=mode)
  df['lpips psp gansformer'] = all_scores.values()
  print(df)

################################################################################
Running on step: 4
################################################################################
Loading dataset
dataloader
loss


100%|██████████| 146/146 [00:01<00:00, 110.85it/s]

Finished with  /content/gdrive/MyDrive/SPRING_2022/EC523/hw/proj/famous/psp_gansformer_attention/4
Average loss is 0.2233+-0.0411
          img  l2 psp base  l2 psp stylegan2  l2 psp gansformer  \
0    1048.jpg     0.055496          0.070964           0.085880   
1    1053.jpg     0.030437          0.029259           0.038544   
2    1056.jpg     0.023546          0.022495           0.037449   
3    1123.jpg     0.020062          0.020812           0.028149   
4    1125.jpg     0.017133          0.017382           0.023909   
..        ...          ...               ...                ...   
141   968.jpg     0.034556          0.030363           0.052896   
142   981.jpg     0.066034          0.072204           0.110232   
143   989.jpg     0.039325          0.039463           0.046755   
144   990.jpg     0.031376          0.028654           0.033184   
145   991.jpg     0.041199          0.039777           0.046131   

     lpips psp base  lpips psp stylegan2  lpips psp gansformer  


In [66]:
df['l2 diff']=df['l2 psp base']-df['l2 psp stylegan2']
df['lpips diff']=df['lpips psp base']-df['lpips psp stylegan2']

#print(df)

newdf = df[df['l2 diff']>0.0]
newdf.reset_index()
#print(newdf)

fin = newdf[newdf['lpips diff']>0.0]
fin.reset_index()
#print(fin)

top = fin[fin['l2 diff']>0.006]
print(top)

          img  l2 psp base  l2 psp stylegan2  l2 psp gansformer  \
23   1501.jpg     0.044437          0.034742           0.055626   
43    183.jpg     0.050115          0.042740           0.064461   
51   2068.jpg     0.053653          0.047468           0.059211   
102    36.jpg     0.057609          0.046650           0.080071   
107   435.jpg     0.059308          0.045888           0.067022   
110  4576.jpg     0.051688          0.045616           0.064139   
111   474.jpg     0.046766          0.039489           0.065791   
128   681.jpg     0.041922          0.033320           0.061292   
137   914.jpg     0.025242          0.018095           0.036129   
138   952.jpg     0.069243          0.059208           0.072516   
140   965.jpg     0.036565          0.030273           0.043173   

     lpips psp base  lpips psp stylegan2  lpips psp gansformer   l2 diff  \
23         0.178448             0.177199              0.245556  0.009695   
43         0.156004             0.143020   